# Correspondence

The notebooks in this folder replicate the experiments as performed for [CNNs on Surfaces using Rotation-Equivariant Features](https://doi.org/10.1145/3386569.3392437).

The current notebook replicates the Correspondence experiments from section `5.2 Comparisons`.

## Imports
We start by importing dependencies.

In [ ]:
# File reading and progressbar
import os.path as osp
import progressbar

# PyTorch and PyTorch Geometric dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader

# Harmonic Surface Networks components
# Layers
from nn import HarmonicResNetBlock, ParallelTransportPool, ParallelTransportUnpool, ComplexLin, ComplexNonLin
# Utility functions
from utils.harmonic import magnitudes
# Transforms
from transforms import (HarmonicPrecomp, VectorHeat, MultiscaleRadiusGraph,
                        NormalizeArea, ScaleMask, FilterNeighbours)

## Settings
Next, we set a few parameters for our network. You can change these settings to experiment with different configurations of the network. Right now, the settings are set to the ones used in the paper.

In [ ]:
# Maximum rotation order for streams
max_order = 1

# Number of rings in the radial profile
n_rings = 2

# Number of filters per block
nf = [16, 32]

# Ratios used for pooling
ratios=[1, 0.25]

# Number of datasets per batch
batch_size = 1

# Remeshed dataset or not:
remeshed = True

# Toggle between original and remeshed FAUST dataset
if remeshed:
    from datasets import FAUSTRemeshed as FAUST
    faust_dir = 'FAUST_5k'
    radii = [0.07, 0.14]
else:
    from datasets import FAUST
    faust_dir = 'FAUST'
    radii = [0.1, 0.2]

## Dataset
To get our dataset ready for training, we need to perform the following steps:
1. Provide a path to load and store the dataset.
2. Define transformations to be performed on the dataset:
    - A transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
    - A transformation that masks the edges and vertices per scale and precomputes convolution components.
3. Assign and load the datasets.

In [ ]:
# 1. Provide a path to load and store the dataset.
# Make sure that you have created a folder 'data' somewhere
# and that you have downloaded and moved the raw datasets there
path = osp.join('data', faust_dir)

# 2. Define transformations to be performed on the dataset:
# Transformation that computes a multi-scale radius graph and precomputes the logarithmic map.
pre_transform = T.Compose((
    MultiscaleRadiusGraph(ratios, radii, loop=True, flow='target_to_source'),
    VectorHeat())
)

# Transformations that mask the edges and vertices per scale and precomputes convolution components.
scale0_transform = T.Compose((
    ScaleMask(0),
    FilterNeighbours(radius=radii[0]),
    HarmonicPrecomp(n_rings, max_order, max_r=radii[0]))
)
scale1_transform = T.Compose((
    ScaleMask(1),
    FilterNeighbours(radius=radii[1]),
    HarmonicPrecomp(n_rings, max_order, max_r=radii[1]))
)

# 3. Assign and load the datasets.
train_dataset = FAUST(path, True, pre_transform=pre_transform)
test_dataset = FAUST(path, False, pre_transform=pre_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
num_nodes = train_dataset[0].num_nodes

## Network architecture
Now, we create the network architecture by creating a new `nn.Module`, `Net`. We first setup each layer in the `__init__` method of the `Net` class and define the steps to perform for each batch in the `forward` method. The following figure shows a schematic of the architecture we will be implementing:

<img src="img/resnet_architecture.png" width="800px" />

Let's get started!

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Pre-transform, linear layer
        self.lin0 = nn.Linear(3, nf[0])

        # Stack 1
        self.resnet_block11 = HarmonicResNetBlock(nf[0], nf[0], max_order, n_rings, prev_order=0)
        self.resnet_block12 = HarmonicResNetBlock(nf[0], nf[0], max_order, n_rings)

        # Pool
        self.pool = ParallelTransportPool(1, scale1_transform)

        # Stack 2
        self.resnet_block21 = HarmonicResNetBlock(nf[0], nf[1], max_order, n_rings)
        self.resnet_block22 = HarmonicResNetBlock(nf[1], nf[1], max_order, n_rings)

        # Stack 3
        self.resnet_block31 = HarmonicResNetBlock(nf[1], nf[1], max_order, n_rings)
        self.resnet_block32 = HarmonicResNetBlock(nf[1], nf[1], max_order, n_rings)
        
        # Unpool
        self.unpool = ParallelTransportUnpool(1)

        # Stack 4
        self.resnet_block41 = HarmonicResNetBlock(nf[1] + nf[0], nf[0], max_order, n_rings)
        self.resnet_block42 = HarmonicResNetBlock(nf[0], nf[0], max_order, n_rings, last_layer=True)

        # Dense final layers
        self.lin3 = nn.Linear(nf[0], 256)
        self.lin4 = nn.Linear(256, num_nodes)


    def forward(self, data):
        # We use xyz positions as input in this notebook
        # in the paper, shot descriptors were used
        x = data.pos
        
        # Linear transformation from input descriptors to nf[0] features
        x = F.relu(self.lin0(x))
        
        # Convert input features into complex numbers
        x = torch.stack((x, torch.zeros_like(x)), dim=-1).unsqueeze(1)

        # Stack 1
        # Select only the edges and precomputed components of the first scale
        data_scale0 = scale0_transform(data)
        attributes = (data_scale0.edge_index, data_scale0.precomp, data_scale0.connection)
        x = self.resnet_block11(x, *attributes)
        x_prepool = self.resnet_block12(x, *attributes)

        # Pooling
        # Apply parallel transport pooling
        x, data, data_pooled = self.pool(x_prepool, data)
        
        # Stack 2
        # Store edge_index and precomputed components of the second scale
        attributes_pooled = (data_pooled.edge_index, data_pooled.precomp, data_pooled.connection)
        x = self.resnet_block21(x, *attributes_pooled)
        x = self.resnet_block22(x, *attributes_pooled)

        # Stack 3
        x = self.resnet_block31(x, *attributes_pooled)
        x = self.resnet_block32(x, *attributes_pooled)

        # Unpooling
        x = self.unpool(x, data)
        # Concatenate pre-pooling x with post-pooling x
        x = torch.cat((x, x_prepool), dim=2)

        # Stack 3
        x = self.resnet_block41(x, *attributes)
        x = self.resnet_block42(x, *attributes)

        # Take radial component from features and sum streams
        x = magnitudes(x, keepdim=False)
        x = x.sum(dim=1)

        x = F.relu(self.lin3(x))
        x = F.dropout(x, training=self.training)
        x = self.lin4(x)
        return F.log_softmax(x, dim=1)

## Training

Phew, we're through the hard part. Now, let's get to training. First, move the network to the GPU and setup an optimizer.

In [ ]:
# We want to train on a GPU. It'll take a long time on a CPU
device = torch.device('cuda')
# Move the network to the GPU
model = Net().to(device)
# Set up the ADAM optimizer with learning rate of 0.0076 (as used in H-Nets)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# The target should be the index of each vertex
target = torch.arange(num_nodes, dtype=torch.long, device=device)

Next, define a training and test function.

In [ ]:
def train(epoch):
    # Set model to 'train' mode
    model.train()

    if epoch == 60:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

    for data in progressbar.progressbar(train_loader):
        # Move training data to the GPU and optimize parameters
        data = data.to(device)
        optimizer.zero_grad()
        F.nll_loss(model(data), target).backward()
        optimizer.step()

Train for 100 epochs.

In [ ]:
for epoch in range(100):
    print('Epoch {}'.format(epoch))
    train(epoch)

## Testing
Finally, we test our model on the test dataset. Setup a test function:

In [ ]:
def test():
    # Set model to 'evaluation' mode
    model.eval()
    correct = 0

    for i, data in progressbar.progressbar(enumerate(test_loader)):
        pred = model(data.to(device)).max(1)[1]
        correct += pred.eq(target).sum().item()
    return correct / (len(test_dataset) * num_nodes)

And return the accuracy on the test set. Note that the exact accuracy for Remeshed FAUST will be very low. The princeton benchmark (see paper, fig 9) gives a better picture of the accuracy of the method in practice.

In [ ]:
test_acc = test()
print('Test: {:.6f}'.format(test_acc))